In [1]:
from __future__ import print_function

In [4]:
from keras.callbacks import LambdaCallback
from keras.models import Model, load_model, Sequential
from keras.layers import Dense, Activation, Dropout, Input, Masking
from keras.layers import LSTM
from keras.utils.data_utils import get_file
from keras.preprocessing.sequence import pad_sequences
import sys, io, random
import numpy as np

### usable functions:

In [14]:
def build_data(text, Tx = 40, stride = 3):
    """
    Create a training set by scanning a window of size Tx over the text corpus, with stride 3.
    Arguments:
    text -- string, corpus of Shakespearian poem
    Tx -- sequence length, number of time-steps (or characters) in one training example
    stride -- how much the window shifts itself while scanning
    
    Returns:
    X -- list of training examples
    Y -- list of training labels
    """
    
    X = []
    Y = []

    for i in range(0, len(text) - Tx, stride):
        X.append(text[i: i + Tx])
        Y.append(text[i + Tx])
    
    print('number of training examples:', len(X))
    
    return X, Y


def vectorization(X, Y, n_x, char_indices, Tx = 40):
    """
    Convert X and Y (lists) into arrays to be given to a recurrent neural network.
    Arguments:
    X -- 
    Y -- 
    Tx -- integer, sequence length
    
    Returns:
    x -- array of shape (m, Tx, len(chars))
    y -- array of shape (m, len(chars))
    """
    
    m = len(X)
    x = np.zeros((m, Tx, n_x), dtype=np.bool)
    y = np.zeros((m, n_x), dtype=np.bool)
    for i, sentence in enumerate(X):
        for t, char in enumerate(sentence):
            x[i, t, char_indices[char]] = 1
        y[i, char_indices[Y[i]]] = 1
        
    return x, y 


def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    out = np.random.choice(range(len(chars)), p = probas.ravel())
    return out
    #return np.argmax(probas)
    
def on_epoch_end(epoch, logs):
    # Function invoked at end of each epoch. Prints generated text.
    None
    #start_index = random.randint(0, len(text) - Tx - 1)
    
    #generated = ''
    #sentence = text[start_index: start_index + Tx]
    #sentence = '0'*Tx
    #usr_input = input("Write the beginning of your poem, the Shakespearian machine will complete it.")
    # zero pad the sentence to Tx characters.
    #sentence = ('{0:0>' + str(Tx) + '}').format(usr_input).lower()
    #generated += sentence
#
    #sys.stdout.write(usr_input)

    #for i in range(400):
"""
        #x_pred = np.zeros((1, Tx, len(chars)))

        for t, char in enumerate(sentence):
            if char != '0':
                x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, temperature = 1.0)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
        
        if next_char == '\n':
            continue
        
    # Stop at the end of a line (4 lines)
    print()
 """   
print("Loading text data...")
text = io.open('lesmian.txt', encoding='utf-8').read().lower()
#print('corpus length:', len(text))

Tx = 40
chars = sorted(list(set(text)))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))
#print('number of unique characters in the corpus:', len(chars))

print("Creating training set...")
X, Y = build_data(text, Tx, stride = 3)
print("Vectorizing training set...")
x, y = vectorization(X, Y, n_x = len(chars), char_indices = char_indices) 
print("Loading model...")
model = load_model('models/model_shakespeare_kiank_350_epoch.h5')


def generate_output():
    generated = ''
    #sentence = text[start_index: start_index + Tx]
    #sentence = '0'*Tx
    usr_input = input("Write the beginning of your poem, the Shakespeare machine will complete it. Your input is: ")
    # zero pad the sentence to Tx characters.
    sentence = ('{0:0>' + str(Tx) + '}').format(usr_input).lower()
    generated += usr_input 

    sys.stdout.write("\n\nHere is your poem: \n\n") 
    sys.stdout.write(usr_input)
    for i in range(400):

        x_pred = np.zeros((1, Tx, len(chars)))

        for t, char in enumerate(sentence):
            if char != '0':
                x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, temperature = 1.0)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()

        if next_char == '\n':
            continue

Loading text data...
Creating training set...
number of training examples: 51031
Vectorizing training set...
Loading model...


In [13]:
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y, batch_size=128, epochs=1, callbacks=[print_callback])

ValueError: Error when checking input: expected input_3 to have shape (40, 38) but got array with shape (40, 73)

In [11]:
generate_output()

Write the beginning of your poem, the Shakespeare machine will complete it. Your input is: We have such a beautiful day today


Here is your poem: 

We have such a beautiful day today me,
love beface parous wook in my wusted nifol,
and nogrong, wrinct farpure erpinl thy mint,
and that the sweens i rementher mu thy lover.
then falme pey a ruke you undude that of shew,
they suh and love you time stome yrit whele,
but dost heatt blest that lais mer the dorcance heart.
 
fwor i spy yy frame mune and my elcid,
and she of dishantouss to thy all cunse bearh.
thour canst with have eve

In [20]:
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, Activation, LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io

path=('lesmian.txt')
with open(path, 'r', encoding='UTF8') as f:
    text=f.read().lower()
print('corpus length:', len(text))

chars=sorted(list(set(text)))
print('total characters: ', len(chars))
char_indices=dict((c,i) for i, c in enumerate(chars))
indices_char=dict((i,c) for i,c in enumerate(chars))

# cut the text into sequences of maxlen characters

maxlen=40
step=3
sentences=[]
next_chars=[]

for i in range(0,len(text)-maxlen, step):
    sentences.append(text[i:i+maxlen])
    next_chars.append(text[i+maxlen])
    print('number of sequences: ',len(sentences))
    
    #vectorization
    x=np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
    y=np.zeros((len(sentences), len(chars)), dtype=np.bool)
    for i, sentence in enumerate(sentences):
        for t, char in enumerate(sentence):
            x[i,t,char_indices[char]]=1
        y[i, char_indices[next_chars[i]]]=1
        
# model
print('building model...')
model=Sequential()
model.add(LSTM(128,input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer=RMSprop(lr=0.1)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

def sample (preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def on_epoch_end(epoch, logs):
    # Prints generated text.
    print('Epoch:', epoch)
    
    start_index = random.randint(0,len(text)-maxlen-1)
    for i in [0.2, 0.5, 1.0, 1.2]:
        print('diversity: ',i)
        generated=''
        sentence=text[start_index: start_index + maxlen]
        generated+=sentence
        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.
            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=60,
          callbacks=[print_callback])    
    



corpus length: 153132
total characters:  73
number of sequences:  1
number of sequences:  2
number of sequences:  3
number of sequences:  4
number of sequences:  5
number of sequences:  6
number of sequences:  7
number of sequences:  8
number of sequences:  9
number of sequences:  10
number of sequences:  11
number of sequences:  12
number of sequences:  13
number of sequences:  14
number of sequences:  15
number of sequences:  16
number of sequences:  17
number of sequences:  18
number of sequences:  19
number of sequences:  20
number of sequences:  21
number of sequences:  22
number of sequences:  23
number of sequences:  24
number of sequences:  25
number of sequences:  26
number of sequences:  27
number of sequences:  28
number of sequences:  29
number of sequences:  30
number of sequences:  31
number of sequences:  32
number of sequences:  33
number of sequences:  34
number of sequences:  35
number of sequences:  36
number of sequences:  37
number of sequences:  38
number of seque

number of sequences:  348
number of sequences:  349
number of sequences:  350
number of sequences:  351
number of sequences:  352
number of sequences:  353
number of sequences:  354
number of sequences:  355
number of sequences:  356
number of sequences:  357
number of sequences:  358
number of sequences:  359
number of sequences:  360
number of sequences:  361
number of sequences:  362
number of sequences:  363
number of sequences:  364
number of sequences:  365
number of sequences:  366
number of sequences:  367
number of sequences:  368
number of sequences:  369
number of sequences:  370
number of sequences:  371
number of sequences:  372
number of sequences:  373
number of sequences:  374
number of sequences:  375
number of sequences:  376
number of sequences:  377
number of sequences:  378
number of sequences:  379
number of sequences:  380
number of sequences:  381
number of sequences:  382
number of sequences:  383
number of sequences:  384
number of sequences:  385
number of se

number of sequences:  677
number of sequences:  678
number of sequences:  679
number of sequences:  680
number of sequences:  681
number of sequences:  682
number of sequences:  683
number of sequences:  684
number of sequences:  685
number of sequences:  686
number of sequences:  687
number of sequences:  688
number of sequences:  689
number of sequences:  690
number of sequences:  691
number of sequences:  692
number of sequences:  693
number of sequences:  694
number of sequences:  695
number of sequences:  696
number of sequences:  697
number of sequences:  698
number of sequences:  699
number of sequences:  700
number of sequences:  701
number of sequences:  702
number of sequences:  703
number of sequences:  704
number of sequences:  705
number of sequences:  706
number of sequences:  707
number of sequences:  708
number of sequences:  709
number of sequences:  710
number of sequences:  711
number of sequences:  712
number of sequences:  713
number of sequences:  714
number of se

number of sequences:  1003
number of sequences:  1004
number of sequences:  1005
number of sequences:  1006
number of sequences:  1007
number of sequences:  1008
number of sequences:  1009
number of sequences:  1010
number of sequences:  1011
number of sequences:  1012
number of sequences:  1013
number of sequences:  1014
number of sequences:  1015
number of sequences:  1016
number of sequences:  1017
number of sequences:  1018
number of sequences:  1019
number of sequences:  1020
number of sequences:  1021
number of sequences:  1022
number of sequences:  1023
number of sequences:  1024
number of sequences:  1025
number of sequences:  1026
number of sequences:  1027
number of sequences:  1028
number of sequences:  1029
number of sequences:  1030
number of sequences:  1031
number of sequences:  1032
number of sequences:  1033
number of sequences:  1034
number of sequences:  1035
number of sequences:  1036
number of sequences:  1037
number of sequences:  1038
number of sequences:  1039
n

number of sequences:  1309
number of sequences:  1310
number of sequences:  1311
number of sequences:  1312
number of sequences:  1313
number of sequences:  1314
number of sequences:  1315
number of sequences:  1316
number of sequences:  1317
number of sequences:  1318
number of sequences:  1319
number of sequences:  1320
number of sequences:  1321
number of sequences:  1322
number of sequences:  1323
number of sequences:  1324
number of sequences:  1325
number of sequences:  1326
number of sequences:  1327
number of sequences:  1328
number of sequences:  1329
number of sequences:  1330
number of sequences:  1331
number of sequences:  1332
number of sequences:  1333
number of sequences:  1334
number of sequences:  1335
number of sequences:  1336
number of sequences:  1337
number of sequences:  1338
number of sequences:  1339
number of sequences:  1340
number of sequences:  1341
number of sequences:  1342
number of sequences:  1343
number of sequences:  1344
number of sequences:  1345
n

...

number of sequences:  29511
number of sequences:  29512
number of sequences:  29513
number of sequences:  29514
number of sequences:  29515
number of sequences:  29516
number of sequences:  29517
number of sequences:  29518
number of sequences:  29519
number of sequences:  29520
number of sequences:  29521
number of sequences:  29522
number of sequences:  29523
number of sequences:  29524
number of sequences:  29525
number of sequences:  29526
number of sequences:  29527
number of sequences:  29528
number of sequences:  29529
number of sequences:  29530
number of sequences:  29531
number of sequences:  29532
number of sequences:  29533
number of sequences:  29534
number of sequences:  29535
number of sequences:  29536
number of sequences:  29537
number of sequences:  29538
number of sequences:  29539
number of sequences:  29540
number of sequences:  29541
number of sequences:  29542
number of sequences:  29543
number of sequences:  29544
number of sequences:  29545
number of sequences:

number of sequences:  29804
number of sequences:  29805
number of sequences:  29806
number of sequences:  29807
number of sequences:  29808
number of sequences:  29809
number of sequences:  29810
number of sequences:  29811
number of sequences:  29812
number of sequences:  29813
number of sequences:  29814
number of sequences:  29815
number of sequences:  29816
number of sequences:  29817
number of sequences:  29818
number of sequences:  29819
number of sequences:  29820
number of sequences:  29821
number of sequences:  29822
number of sequences:  29823
number of sequences:  29824
number of sequences:  29825
number of sequences:  29826
number of sequences:  29827
number of sequences:  29828
number of sequences:  29829
number of sequences:  29830
number of sequences:  29831
number of sequences:  29832
number of sequences:  29833
number of sequences:  29834
number of sequences:  29835
number of sequences:  29836
number of sequences:  29837
number of sequences:  29838
number of sequences:

number of sequences:  30097
number of sequences:  30098
number of sequences:  30099
number of sequences:  30100
number of sequences:  30101
number of sequences:  30102
number of sequences:  30103
number of sequences:  30104
number of sequences:  30105
number of sequences:  30106
number of sequences:  30107
number of sequences:  30108
number of sequences:  30109
number of sequences:  30110
number of sequences:  30111
number of sequences:  30112
number of sequences:  30113
number of sequences:  30114
number of sequences:  30115
number of sequences:  30116
number of sequences:  30117
number of sequences:  30118
number of sequences:  30119
number of sequences:  30120
number of sequences:  30121
number of sequences:  30122
number of sequences:  30123
number of sequences:  30124
number of sequences:  30125
number of sequences:  30126
number of sequences:  30127
number of sequences:  30128
number of sequences:  30129
number of sequences:  30130
number of sequences:  30131
number of sequences:

number of sequences:  30390
number of sequences:  30391
number of sequences:  30392
number of sequences:  30393
number of sequences:  30394
number of sequences:  30395
number of sequences:  30396
number of sequences:  30397
number of sequences:  30398
number of sequences:  30399
number of sequences:  30400
number of sequences:  30401
number of sequences:  30402
number of sequences:  30403
number of sequences:  30404
number of sequences:  30405
number of sequences:  30406
number of sequences:  30407
number of sequences:  30408
number of sequences:  30409
number of sequences:  30410
number of sequences:  30411
number of sequences:  30412
number of sequences:  30413
number of sequences:  30414
number of sequences:  30415
number of sequences:  30416
number of sequences:  30417
number of sequences:  30418
number of sequences:  30419
number of sequences:  30420
number of sequences:  30421
number of sequences:  30422
number of sequences:  30423
number of sequences:  30424
number of sequences:

KeyboardInterrupt: 